<a href="https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Evaluating_Big_Bird_on_TriviaQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluation of a model using 🤗nlp

*This notebook shows how to evaluate **BigBird** on **TriviaQA** .*

- The [`datasets`](https://github.com/huggingface/datasets) library allows simple and intuitive access to nlp datasets and metrics.

- **Big Bird** is a transformer-based model for long-range sequence modeling introduced by *Manzil Zaheer, et. al* (see paper [here](https://arxiv.org/abs/2007.14062)) and can now be accessed via Transformers via the [TODO: docs]().

- **TriviaQA** is a reading comprehension dataset containing question-answer-evidence triplets (see paper here [here](https://homes.cs.washington.edu/~eunsol/papers/acl17jcwz.pdf)).

We will evaluate a pretrained `BigBirdForQuestionAnswering` model on the *validation* dataset of **TriviaQA**. Along the way, this notebook will show you how `datasets` can be used for effortless preprocessing of data and analysis of the results.

Alright! Let's start by installing the `datasets` library and loading *TriviaQA*. 

### Installs and Imports

Install `transformers` & `datasets`.

In [3]:
%%capture
!pip install datasets==1.5.0
!pip install git+https://github.com/vasudevgupta7/transformers.git@add_big_bird # TODO: replace with new pip version eventually

import datasets
import torch

### Data Preprocessing & Cleaning 

The total *TriviaQA* dataset has a size of 17 GB once processed.
Downloading and preprocessing the dataset will take around *15 minutes*. ☕
Afterwards the data is serialized in *Arrow* format, so that subsequent call can reuse the processed dataset.

**Note**: If you want to evaluate *BigBird* on the full validation set this notebook will run for *ca.* 6 hours ⌛. On 5% of the data it takes around 30 minutes.



In [4]:
validation_dataset = datasets.load_dataset("trivia_qa", "rc", split="validation[:5%]")  # remove [:5%] to run on full validation set

Dataset trivia_qa downloaded and prepared to /root/.cache/huggingface/datasets/trivia_qa/rc/1.1.0/c1b17d80bfdd2f43b13e4b5741847f704115379ecebcb21e0917d2e8ddaa1ff1. Subsequent calls will reuse this data.


First, let's get an overview of the dataset 🧐.

In [5]:
validation_dataset

Dataset({
    features: ['question', 'question_id', 'question_source', 'entity_pages', 'search_results', 'answer'],
    num_rows: 933
})

Alright, there are ~19K data samples in the validation step.

Next, let's check out the datatset's structure.

In [6]:

validation_dataset.info.features

{'answer': {'aliases': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
  'matched_wiki_entity_name': Value(dtype='string', id=None),
  'normalized_aliases': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
  'normalized_matched_wiki_entity_name': Value(dtype='string', id=None),
  'normalized_value': Value(dtype='string', id=None),
  'type': Value(dtype='string', id=None),
  'value': Value(dtype='string', id=None)},
 'entity_pages': Sequence(feature={'doc_source': Value(dtype='string', id=None), 'filename': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'wiki_context': Value(dtype='string', id=None)}, length=-1, id=None),
 'question': Value(dtype='string', id=None),
 'question_id': Value(dtype='string', id=None),
 'question_source': Value(dtype='string', id=None),
 'search_results': Sequence(feature={'description': Value(dtype='string', id=None), 'filename': Value(dtype='string', id=None), 'rank': Value(dtype='int32', 

Alright, quite a lot of entries here! For Questions Answering, all we need is the *question*, the *context* and the *answer*. 

The **question** is a single entry, so we keep it.

Because *BigBird* was trained on the Wikipedia part of *TriviaQA*, we will use `validation_dataset["entity_pages"]["wiki_context"]` as our **context**. 

We can also see that there are multiple entries for the **answer**. In this use case, we define a correct output of the model as one that is one of the answer aliases `validation_dataset["answer"]["aliases"]`. Lastly, we also keep `validation_dataset["answer"]["normalized_value"]`. All other columns can be disregarded. 

Let's write the corresponding `.map(...)` function

In [7]:
def format_dataset(example):
    # the context might be comprised of multiple contexts => me merge them here
    example["context"] = " ".join(("\n".join(example["entity_pages"]["wiki_context"])).split("\n"))
    example["targets"] = example["answer"]["aliases"]
    example["norm_target"] = example["answer"]["normalized_value"]
    return example

and apply it so that the dataset is put into the format as defined above.

In [8]:
validation_dataset = validation_dataset.map(format_dataset, remove_columns=["search_results", "question_source", "entity_pages", "answer", "question_id"])

Now, let's check out a couple of data samples. 

For this we define a function that picks `num_examples` random data samples.

In [9]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

Let's run it.

In [10]:
show_random_elements(validation_dataset, num_examples=3)

Great 🙂 - that's the structure that we wanted! Looking at the examples should also make it clear how large the context of the model can be! Also note that some samples can have an empty context. For those samples it is impossible to extract the correct answer from the context since the context is empty. 

We filter out those data samples for which we can use the convenient `.filter()` function of `datasets` again.

In [11]:
validation_dataset = validation_dataset.filter(lambda x: len(x["context"]) > 0)
# check out how many samples are left
validation_dataset

Dataset({
    features: ['context', 'norm_target', 'question', 'targets'],
    num_rows: 531
})

Looks like more or less half of our examples have no context and are now filtered out. Let's think about the evaluation on *BigBird* now. 

*BigBird* is able to process inputs of up to a length of **4096** tokens, has a vocab size of *ca.* 50K tokens, and makes use of the [**sentencepiece**](https://github.com/google/sentencepiece) tokenizer. 

Given this information, we can assume that a single token of *BigBird*'s vocabulary represents roughly 4 characters on average. 

To better understand how important it is to take the full context into account, we create a second `"shorter"` dataset. This `short_validation_dataset` only consists of data samples that do **not** exceed 4 * 4096 characters, which corresponds *ca.* to *BigBird*'s maximum length of 4096 tokens.

Again we can apply the convenient `.filter()` function.

In [12]:
short_validation_dataset = validation_dataset.filter(lambda x: (len(x['question']) + len(x['context'])) < 4 * 4096)
short_validation_dataset


Dataset({
    features: ['context', 'norm_target', 'question', 'targets'],
    num_rows: 127
})

Interesting! We can see that only 4159 of the 16504 examples have less than $4 \times 4096 = 16384$ characters...

Most examples seem to have a very long context which will have to be cut to BigBird's maximum length when evaluating the full dataset.

Great, we have now preprocessed the complete dataset `validation_dataset` and a shorter version `short_validation_dataset` that only consists of data samples $< 4096$ tokens.

### Evaluation

It's time to evaluate *BigBird* on *TriviaQA* 🚀. `BigBirdTokenizer` requires the `sentencepiece` library, so let's install this first.

In [13]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 6.6MB/s 


Let's import the pretrained `BigBirdForQuestionAnswering` model. For more details on `BigBirdForQuestionAnswering`, see [TODO: here]().

In [19]:
from transformers import BigBirdTokenizer, BigBirdForQuestionAnswering

tokenizer = BigBirdTokenizer.from_pretrained("google/bigbird-base-trivia-itc")
model = BigBirdForQuestionAnswering.from_pretrained("google/bigbird-base-trivia-itc").to("cuda")


Next, we will write the evaluation function. Google's official evaluation scripts for *TriviaQA* includes many improvements to make sure that:

- multiple aliases of **both** the model's prediction **and** the label are created. This way it can be assured that a correct prediction that only differs in format to the label is indeed classified as being correct. For this, we will write an `expand_to_aliases` function which will normalise the targets and prediction.
- the model returns a *non-empty* answer. This can be achieved by using the `top_k` scores returned by model to filter out scores which would lead to an emtpy answer.

Let's first define the `expand_to_aliases` function.

In [28]:
PUNCTUATION_SET_TO_EXCLUDE = set(''.join(['‘', '’', '´', '`', '.', ',', '-', '"']))

def get_sub_answers(answers, begin=0, end=None):
  return [" ".join(x.split(" ")[begin:end]) for x in answers if len(x.split(" ")) > 1]

def expand_to_aliases(given_answers, make_sub_answers=False):
  if make_sub_answers:
    # if answers are longer than one word, make sure a predictions is correct if it coresponds to the complete 1: or :-1 sub word
    # *e.g.* if the correct answer contains a prefix such as "the", or "a"
    given_answers = given_answers + get_sub_answers(given_answers, begin=1) + get_sub_answers(given_answers, end=-1)
  answers = []
  for answer in given_answers:
    alias = answer.replace('_', ' ').lower()
    alias = ''.join(c if c not in PUNCTUATION_SET_TO_EXCLUDE else ' ' for c in alias)
    answers.append(' '.join(alias.split()).strip())
  return set(answers)

Here we use a RoBERTa-like model for question answering which means that the answer is not generated auto-regressively by the model, but rather the model has to extract the correct answer from the context. It does so by predicted the start index of the answer as well as the end index and therefore returns two logit vectors at evaluation:

1. `start_scores` of size $\text{batch_size} \times \text{sequence_length}$ and
2. `end_scores` of size $\text{batch_size} \times \text{sequence_length}$, with

$\text{sequence_length}$ being the sum of the question length and the context length. Each value in `start_scores` and `end_scores` is therefore proportional to the probability that probability that its corresponding index is the start (*resp.* end) index of the answer. The answer can then be computed as the subarray of the tokenized input question + input context: 

```python
predicted_answer = tokenized_question_and_context[best_start_index: best_end_index + 1]
``` 

Now instead of just taking the most likely start and end index, we can compute the $\text{top_k}^2$ best start and end index combinations and pick the most likely combination that is **valid**. We define a **valid** combination as one where the start index is smaller than the end index and where $\text{end_index} - \text{start_index} < \text{max_size}$.

Alright quite some theory here - let's write up the function.

In [21]:
def get_best_valid_start_end_idx(start_scores, end_scores, top_k=1, max_size=100):
    best_start_scores, best_start_idx = torch.topk(start_scores, top_k)
    best_end_scores, best_end_idx = torch.topk(end_scores, top_k)

    widths = best_end_idx[:, None] - best_start_idx[None, :]
    mask = torch.logical_or(widths < 0, widths > max_size)
    scores = (best_end_scores[:, None] + best_start_scores[None, :]) - (1e8 * mask)
    best_score = torch.argmax(scores).item()

    return best_start_idx[best_score % top_k], best_end_idx[best_score // top_k]

Alright, finally we can write the evaluation function. First, we tokenize the `question` and `context` and make sure that the maximum length does not exceed the model's `max_position_embeddings` of 4096 tokens.
Then, we pass the concatenated question and context token IDs to the model to retrieve the `start_scores` and `end_scores`.

As explained earlier, using just the most likely start and end idx could lead to invalid answers, so we apply the `get_best_valid_start_end_idx(...)` function to retrieve the best *valid* start and end token ids.
Given those ids, we can then retrieve the actual predicted answer string.
Finally, we create normalized aliases for both the predicted answer and the labels. If there is an overlap between all "extended" predictions and labels then we have a match.

In [26]:
def evaluate(example):
    # encode question and context so that they are seperated by a tokenizer.sep_token and cut at max_length
    encoding = tokenizer(example["question"], example["context"], return_tensors="pt", max_length=4096, padding="max_length", truncation=True)
    input_ids = encoding.input_ids.to(device)

    with torch.no_grad():
        start_scores, end_scores = model(input_ids=input_ids).to_tuple()

    start_score, end_score = get_best_valid_start_end_idx(start_scores[0], end_scores[0], top_k=8, max_size=16)

    # Let's convert the input ids back to actual tokens 
    all_tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"][0].tolist())
    answer_tokens = all_tokens[start_score: end_score + 1]

    example["output"] = tokenizer.decode(tokenizer.convert_tokens_to_ids(answer_tokens))
    #.replace('"', '')  # remove space prepending space token and remove unnecessary '"'

    answers = expand_to_aliases(example["targets"], make_sub_answers=True)
    predictions = expand_to_aliases([example["output"]])

    # if there is a common element, it's a match
    example["match"] = len(list(answers & predictions)) > 0

    return example


To begin with, we are interested in the performance of the model on short and long samples.
Let's evaluate the model on tnhe `short_validation_dataset`. 

**Note**: This function is expected to run for *ca.* 1h if the full dataset is used ⌛

In [29]:
results_short = short_validation_dataset.map(evaluate)

Alright, let's see the result. We report all results as "Exact Match" which is just the ration of correctly answered questions to all questions.

In [40]:
print("Exact Match (EM): {:.2f}".format(100 * sum(results_short['match'])/len(results_short)))

Exact Match (EM): 82.68


105/127 = 82.68 => pretty good 🔥. Let's take a look at the wrong examples.
For this we first filter out the correct examples and then print them out.

In [33]:
wrong_results = results_short.filter(lambda x: x['match'] is False)
print(f"\nWrong examples: ")
print_out = wrong_results.map(lambda x, i: print(f"{i} - Output: {x['output']} - Target: {x['norm_target']}"), with_indices=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/trivia_qa/rc/1.1.0/c1b17d80bfdd2f43b13e4b5741847f704115379ecebcb21e0917d2e8ddaa1ff1/cache-cd4def4df903b397.arrow



Wrong examples: 
0 - Output: Mainland - Target: shetland


0 - Output: Mainland - Target: shetland
1 - Output:  - Target: film making
2 - Output: tartar sauce - Target: puttanesca
3 - Output: collapsible - Target: baby buggy
4 - Output: five - Target: 3
5 - Output: Martin Austin Ruane - Target: martin ruane
6 - Output: Kipps: The Story of a Simple Soul - Target: kipps
7 - Output:  - Target: corset
8 - Output: 102 - Target: 8
9 - Output: heart failure - Target: high blood pressure
10 - Output: Potatoes - Target: onions
11 - Output: citric acid - Target: tartaric acid
12 - Output: Johnny Hooker - Target: henry gondorf
13 - Output: Roald Amundsen - Target: edmond hillary
14 - Output: Sebastian - Target: jasper
15 - Output: George III - Target: george
16 - Output: 1938 - Target: 13
17 - Output: Gary Lewis - Target: gary lewis and playboys
18 - Output: collapsible - Target: baby buggy
19 - Output:  - Target: florence henderson
20 - Output:  - Target: presley
21 - Output: Belle Elmore - Target: his wife



Ok! Except for the wrong answer no. 15: *George III* vs. *george*, where the model arguably gave a correct answer, incorrectly rated model predictions are in fact very much wrong.

We also want to evaluate the `BigBird` on the full validation set. Let's run it.

**Note**: This function is expected to run for ca. 4h if the full dataset is used ⌛⌛⌛⌛. In this case, you should make sure that your training doesn't stop due to inactivity. A simple hack to prevent this is to paste the following code into the console of this tab (right mouse click -> inspect -> Console tab and insert code).

```javascript
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
```

In [34]:
results = validation_dataset.map(evaluate)

Finally, let's print out the score.

In [39]:
print("Exact Match (EM): {:.2f}".format(100 * sum(results['match'])/len(results)))

Exact Match (EM): 62.15


Here, we now see a slight degradation, which is expected since the added data samples all have a longer context than the model can handle. It is still a very good score though.

**Note**: *BigBird* reached a new SOTA on TriviaQA - see Table 2 in [paper](https://arxiv.org/pdf/2007.14062.pdf). Running the notebook on the full validation set should give you an Exact Match score (EM) of TODO: Fill in. 
This score is somewhat close to the reported EM of **75.77** and F-score of **79.5**. The reported EM is still significantly better because the [official evaluation script](https://github.com/tensorflow/models/tree/f540d472d5cdaa701615e7ceb138ef66f3da2770/official/nlp/projects/triviaqa) contain more improvements, such as splitting long contexts, etc...

🤗 🤗 **Finish** 🤗🤗